In [1]:
from htmd import *
# so our plots remain in the notebook
%pylab inline
# utilities
import re

Please cite. HTMD: High-Throughput Molecular Dynamics for Molecular Discovery, J. Chem. Theory Comput., 2016, 12 (4), pp 1845-1852. http://pubs.acs.org/doi/abs/10.1021/acs.jctc.6b00049


You are on the latest HTMD version (unpackaged).
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


# Parameterize Benzamidine with HTMD

by [João M. Damas](https://github.com/j3mdamas)

![](http://bdb2.ucsd.edu/data/jpeg/tenK0/BindingDB_772.png)

## Why the need to parameterize?

![](https://static1.squarespace.com/static/52f2cd27e4b00e69a8f53575/t/545c0968e4b00e059c13b0d5/1415317865874/force_field.jpg)

* All molecules we want to simulate need parameters to describe their physical behaviour
* Many biomolecules are constituted by building blocks (ex: aminoacids) and their parameters are already available (through forcefields)
* When we use forcefields, we are basically fetching from a library of parameters and assigning them to our molecules
* What if the molecule we want to simulate does not have parameters defined yet (example: a ligand that binds to a protein)? Then, we need to parameterize them ourselves!

## Parameterization made easy with HTMD

HTMD has an in-built parameterization tool called `parameterize`. It has been going through some modifications lately, since it was a separate package and we have been integrating it into HTMD. Please be sure of having an HTMD version 1.0.26 or greater.

## Let's first look at the ligand: Benzamidine

Using what we have learned previouly, we can look at the compound we will be parameterizing, **benzamidine**. Let's first load it, and then visualize it:

In [4]:
ben = Molecule('3PTB')
ben.filter('resname BEN')
ben.view(viewer='ngl')

2016-06-07 19:39:38,188 - htmd.molecule.molecule - INFO - Using local copy for 3PTB: /home/joao/maindisk/software/repos/multiscalelab/htmd/htmd/data/pdb/3ptb.pdb
2016-06-07 19:39:38,248 - htmd.molecule.molecule - INFO - Removed 1692 atoms. 9 atoms remaining in the molecule.


As you can see, it is a very simple molecule, with just 9 atoms. Since `parameterize` does quantum calculations, which are computationally demanding, the more particles and torsions the compound we want to parameterize has, the longer it takes.

## Import from Parameterize

So let's start to parameterize our ethanol. First, we need to some import the `Configuration` and `Parameterisation` classes from the `parameterize` module:

In [ ]:
from htmd.parameterize import Configuration, Parameterisation

## Configure our Parameterization

`parameterize` works in a different way from other HTMD modules, since it can be called from outside python and it mainly runs a lot of routines (for QM calculation) which are managed by the python environment.

First, we need to configure our parameterization run using the `Configuration` class:

In [ ]:
config = Configuration()
config.FileName = "./ethanol.mol2"
config.JobName = "ethanol-1"
config.NetCharge = "0"
#config.NCORES = 4 # if one wants to define them, otherwise it will use all resources available
#config.MEMORY = 4 #

## Let's Parameterize!

In [ ]:
param = Parameterisation(config=config)

## `Parameterize` structure and organization

As said before, `parameterize` works a bit different from HTMD, also in the way it stores the information it produces. Since a ligand parameterization can take days, the several steps of `parameterize` are "checkpointed" inside a directory:

In [ ]:
param.directory

We can list what is inside that directory:

In [ ]:
%ls /home/joao/.htmd/gaamp/ethanol-1

## `Parameterize` Runs Management

The structure of `parameterize` allows resuming previous jobs (not covered today) and multiple runs. As such, each `parameterize` run must have a different name. You can list which ones are already taken:

In [ ]:
Parameterisation.listJobs()

If you want to delete a run, you can use:

In [ ]:
#Parameterisation.deleteJob('ethanol-1') # uncomment if you want to delete the job

## Back to Ethanol: Minimized Structure

During `parameterize`, the molecule is optimized/minimized using quantum mechanics. We can look at the end result molecule:

In [ ]:
min_ethanol = param.getMolecule()
min_ethanol.view(viewer='ngl')

Look at how the alcohol group hydrogen moved.

## Torsions: MM vs. QM vs. Parameterize

We can see which dihedrals were parameterized, either by the atom indexes:

In [ ]:
param.getDihedralIndexes()

Or by visualizing the dihedral (using a dihedral index):

In [ ]:
param.showDihedral(0)

## Plot Dihedral Energetics

We can plot the dihedrals and compare the initial MM, QM, and parameterized MM:

In [ ]:
(torsionfiles, torsionnames) = param.plotDihedrals(show=False)

In [ ]:
print(torsionfiles, torsionnames)

In [ ]:
for i  in range(len(torsionfiles)):
    shutil.copyfile(torsionfiles[i], "torsion-potential-" + re.sub(" ", "-", torsionnames[i]) + ".svg" )

## Get the Parameter files

In [ ]:
paramfiles = param.getParameters()
shutil.copyfile(paramfiles['RTF'], "ethanol.rtf")
shutil.copyfile(paramfiles['PRM'], "ethanol.prm")
shutil.copyfile(paramfiles['PDB'], "ethanol-final.pdb")

These structure and parameter files are then compatible to use with CHARMM topologies and are ready to import into HTMD and simulate. You can see the protein ligand tutorial, where the .prm and .rtf files of the ligand are given to  `charmm.build`

As an extra, you can try and parameterize benzamidine at home and check the more complex results. Note: it will take much longer...